## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [ ]:
import string 
import numpy as np
import json

with open('recipes.json') as file:
    recipes= json.load(file)

In [ ]:
#tokenisation function to take care of punctutation and digits

def tokenisation(item):
    x=string.punctuation+string.digits
    y=' '*len(x)
    table=str.maketrans(x,y)
    item=str(item)
    item=item.translate(table)
    item=' '.join(word for word in item.split() if len(word)>=3)
    item=item.lower()
    item=item.split()
   
    return item


In [ ]:
tokenisation(recipes)


In [ ]:
def get_token(dictionary):
    
    keys=['title','categories','ingredients','directions']
    weights={'title':8,'categories':4,'ingredients':2,'directions':1}
    list1=[]
    for key,values in dictionary.items():
        if key in keys:
            tokens=tokenisation(str(values))
            list1+=[tokens]*weights[key]
    search_list=[token for sub in list1 for token in sub]
    
    return search_list

In [ ]:
#invert indexing
inverted_index={}

for i,recipe in enumerate(recipes):
    recipe['tokens']=get_token(recipe)
    for token in recipe['tokens']:
        if token not in inverted_index:
            inverted_index[token]=[]
        inverted_index[token]+=[i]
    if 'rating' not in recipe.keys():
        recipe['rating']=0
        
#simple ordering function

simple=np.zeros(len(recipes))

for i,recipe in enumerate(recipes):
    for key in ['directions','ingredients']:
        if key not in recipe or len(recipe[key])==1:
            break
    else:
#         print("hello")
        simple[i]=len(recipe['directions'])*len(recipe['ingredients'])
        
        
        
#healthy ordering fucntion

cost_functions= lambda n,cal,pro,f : np.fabs(cal-510*n.reshape(-1,1))/ 510 +\
                                     np.fabs(pro-18*n.reshape(-1,1)) * (2/18) +\
                                     np.fabs(f-150*n.reshape(-1,1)) * (4/150)

calories=np.zeros(len(recipes))
protein=np.zeros(len(recipes))
fat=np.zeros(len(recipes))

for i,recipe in enumerate(recipes):
    for key in ['calories','protein','fat']:
        if key not in recipe:
            recipe[key]=0
            
    
    calories[i]=recipe['calories']
    protein[i]=recipe['protein']
    fat[i]=recipe['fat']

healthy=cost_functions(np.arange(1,10),calories,protein,fat).min(axis=0)
unvalid=(calories==0)|(protein==0)|(fat==0)
healthy[unvalid]=0
        
    
        

In [ ]:
#function creating a list of all the recipes containing query words

def query_index(query):
    index_in_rec=set(inverted_index[query[0]])
    if len(query)>1:
        for token in query[1:]:
            index_in_rec=set.intersection(index_in_rec,set(inverted_index[token]))
#         print(recipes[list(index_in_rec)])
    index_in_rec=np.array(list(index_in_rec))
    
    
    
    
    return index_in_rec

In [ ]:
def order_normal(query,idx):
    
    score=np.array([sum([recipes[i]['tokens'].count(token) for token in query])+recipes[i]['rating'] for i in idx])
    #print(score.argsort())
    order1=score.argsort()[::-1]
    #print(order1)
    scored_rec=[recipes[i]['title'] for i in idx[order1]]
 
    return scored_rec
    

In [ ]:
def order_simple(idx):
    
    #print(simple[idx])
    valid=simple[idx]>0
    #print(valid)
    order=simple[idx]
    #print(order)
    order=order[valid]
    #print(order)
    order=order.argsort()
    #print(order)
    scored_rec=[recipes[i]['title'] for i in idx[valid][order]]
#     print(simple[idx][valid][simple[idx][valid].argsort()][:10])
    #print(simple[order])
    return scored_rec


In [ ]:
def order_healthy(idx):
    
    valid=healthy[idx]>0
    order=healthy[idx][valid].argsort()
    scored_rec=[recipes[i]['title'] for i in idx[valid][order]]
   
    
    return scored_rec
    

In [ ]:
def search(query, ordering='normal', count=10):
    try:
    
        query=tokenisation(query)
        matched_indices=query_index(query)
    
        if ordering=='normal':
            print1=order_normal(query,matched_indices)
        elif ordering=='simple':
            print1=order_simple(matched_indices)
            #print(print1)
            #print(score_rec)
        elif ordering=='healthy':
            print1=order_healthy(matched_indices)
        if len(print1)<count:
            print1=print(*print1,sep='\n')
        else:
            print1=print(*print1[:count],sep='\n')
        
        return print1
    except:
        pass

search('cheese cake', 'normal', 20)
          